In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

In [4]:
import os
from dotenv import load_dotenv

In [5]:
load_dotenv()
api_key = os.environ.get('OPENAI_API_KEY')

In [9]:
chat = ChatOpenAI(
    openai_api_key=api_key, 
    model_name="gpt-3.5-turbo",
    temperature = 0.1)

In [13]:
template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}",
                                        )

prompt = template.format(country_a = "Mexico", country_b = "Thailand")

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [16]:

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
])

prompt = template.format_messages(
    language="Greek",
    name = "aa",
    country_a = "Mexico", 
    country_b = "Thailand"

)

In [17]:
chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Το όνομά μου είναι Γεωργία. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [18]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()

p.parse("Hello, how, are, you")


['Hello', 'how', 'are', 'you']

In [20]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a list of max {max_items}. Do NOT reply with anthying else."),
    ("human", "{question}"),
])

prompt = template.format_messages(
    max_items=10,
    question = "What are the planets?"
    
)

chat.predict_messages(prompt)

AIMessage(content='1. Mercury\n2. Venus\n3. Earth\n4. Mars\n5. Jupiter\n6. Saturn\n7. Uranus\n8. Neptune\n9. Pluto')

In [21]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

['1. Pikachu\n2. Charizard\n3. Bulbasaur\n4. Squirtle\n5. Jigglypuff']

In [2]:
# Ch4. FewShotPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

In [7]:
chat = ChatOpenAI(
    openai_api_key=api_key, 
    model_name="gpt-3.5-turbo",
    temperature = 0.1,
    streaming = True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
    )

In [8]:
t = PromptTemplate.from_template("What is the capital of {country}?")
t.format(country="France")


'What is the capital of France?'

In [9]:
t = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"],
)

t.format(country="France")

'What is the capital of France'

In [10]:
examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

chat.predict("What do you know about France?")

France is a country located in Western Europe, known for its rich history, culture, and cuisine. It is the largest country in the European Union by land area and the third-largest in Europe overall. The capital city is Paris, which is famous for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.

France is a popular tourist destination, attracting millions of visitors each year to its beautiful countryside, historic cities, and world-renowned vineyards. The country is also known for its fashion industry, with Paris being considered one of the fashion capitals of the world.

French cuisine is highly regarded internationally, with dishes such as croissants, baguettes, cheese, and wine being staples of French gastronomy. The country is also famous for its fine dining restaurants and Michelin-starred chefs.

France has a long and influential history, having been a major player in European politics and culture for centuries. It was a colonial power with 

'France is a country located in Western Europe, known for its rich history, culture, and cuisine. It is the largest country in the European Union by land area and the third-largest in Europe overall. The capital city is Paris, which is famous for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.\n\nFrance is a popular tourist destination, attracting millions of visitors each year to its beautiful countryside, historic cities, and world-renowned vineyards. The country is also known for its fashion industry, with Paris being considered one of the fashion capitals of the world.\n\nFrench cuisine is highly regarded internationally, with dishes such as croissants, baguettes, cheese, and wine being staples of French gastronomy. The country is also famous for its fine dining restaurants and Michelin-starred chefs.\n\nFrance has a long and influential history, having been a major player in European politics and culture for centuries. It was a colonial powe

In [12]:
example_template = """
    Human: {question}
    AI: {answer}

"""

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt =example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]

)

chain = prompt | chat

chain.invoke({
    "country":"Germany"
})

AI:
I know this:
Capital: Berlin
Language: German
Food: Bratwurst and Sauerkraut
Currency: Euro

AIMessageChunk(content='AI:\nI know this:\nCapital: Berlin\nLanguage: German\nFood: Bratwurst and Sauerkraut\nCurrency: Euro')

In [13]:
# FewShotChatMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate



examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})


        I know this:
        Capital: Bangkok
        Language: Thai
        Food: Pad Thai and Tom Yum
        Currency: Thai Baht
        

AIMessageChunk(content='\n        I know this:\n        Capital: Bangkok\n        Language: Thai\n        Food: Pad Thai and Tom Yum\n        Currency: Thai Baht\n        ')

In [14]:
# LengthBasedExampleSelector

from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")
'Human: What do you know about Italy?\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

'Human: What do you know about Italy?\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [15]:
# Serialization and Composition

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)


Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothin' like a good meal to keep me strength up on the high seas! Arrr!

AIMessageChunk(content="Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothin' like a good meal to keep me strength up on the high seas! Arrr!")

In [16]:
# Caching
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))


chat.predict("How do you make italian pasta")

To make Italian pasta, you will need the following ingredients:

- 2 cups of all-purpose flour
- 2 large eggs
- Pinch of salt

Here is a step-by-step guide to making Italian pasta:

1. On a clean work surface, pour the flour and create a well in the center.
2. Crack the eggs into the well and add a pinch of salt.
3. Using a fork, gradually mix the eggs into the flour until a dough forms.
4. Knead the dough for about 10 minutes until it becomes smooth and elastic.
5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.
6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.
7. Cut the dough into your desired shape, such as fettuccine or spaghetti.
8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes, or until al dente.
9. Drain the pasta and toss with your favorite sauce or toppings.

Enjoy your homemade Italian pasta!

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes, or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

In [17]:
chat.predict("How do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes, or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

In [18]:
# Serialization
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback


with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, "\n")
    print(b, "\n")
    print(usage)

Ingredients:
- 1 cup of rice
- 1 cup of water
- 1 tablespoon of nuruk (fermentation starter)
- 1 tablespoon of yeast

Instructions:
1. Rinse the rice under cold water until the water runs clear.
2. In a large pot, combine the rice and water and bring to a boil. Reduce heat to low and simmer for 20 minutes, or until the rice is cooked.
3. Remove the pot from heat and let it cool to room temperature.
4. In a separate bowl, mix the nuruk and yeast with a little bit of warm water to form a paste.
5. Add the paste to the cooled rice and mix well.
6. Cover the pot with a clean cloth and let it ferment in a warm, dark place for 3-4 days.
7. After fermentation, strain the mixture through a cheesecloth to remove any solids.
8. Transfer the liquid to a clean bottle and store in the refrigerator.
9. Serve chilled and enjoy your homemade soju!Ingredients:
- 4 cups all-purpose flour
- 1 packet active dry yeast
- 1 1/2 cups warm water
- 2 tablespoons sugar
- 2 teaspoons salt
- 2 tablespoons olive oi

In [19]:
from langchain.llms.openai import OpenAI

chat = OpenAI(temperature=0.1, max_tokens=450, model="gpt-3.5-turbo-16k")
chat.save("model.json")

In [ ]:
# model 불러오기
from langchain.llms.loading import load_llm
chat = load_llm("model.json")